In [1]:
!nvidia-smi

Fri May 20 12:48:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
#设置路径
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [3]:
! pip install torch==1.6.0

     |████████████████████████████████| 748.8 MB 18 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.


In [4]:
! pip install transformers==4.0.1

     |████████████████████████████████| 1.4 MB 5.3 MB/s 
     |████████████████████████████████| 880 kB 39.5 MB/s 
     |████████████████████████████████| 2.9 MB 34.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=88e3c925fe777b989c60c4292e038fe8d5ddc5e130fbc6ef0169ef41abaa96ac
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [5]:
! pip install torchvision == 0.7.0

ERROR: Invalid requirement: '=='


In [6]:
import random
import numpy as np
import torch
config = {
        'train_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/train.json',
        'dev_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/dev.json',
        'test_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/test.json',
        'output_path': '.',
        'model_path': '/content/drive/MyDrive/Colab Notebooks/dataset/BERT_model',
        'batch_size': 64,
        'num_epochs': 1,
        'max_seq_len': 64,
        'decay': 0.99,
        'kd_coeff': 1.0,
        'learning_rate': 2e-5,
        'warmup_ratio': 0.05,
        'weight_decay': 0.01,
        'use_bucket': True,
        'bucket_multiplier': 200,
        'device': 'cuda',
        'n_gpus': 0,
        'logging_step': 300,
        'ema_start_step': 500,
        'ema_start': False,
        'seed': 2022
    }

if not torch.cuda.is_available():
    config['device'] = 'cpu'
else:
    config['n_gpus'] = torch.cuda.device_count()
    config['batch_size'] *= config['n_gpus']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

2022

In [7]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(config['model_path'])

In [8]:
def build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
  inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens = True, return_token_type_ids =True, return_attention_mask = True)

  inputs['input_ids'].append(inputs_dict['input_ids'])
  inputs['token_type_ids'].append(inputs_dict['token_type_ids'])
  inputs['attention_mask'].append(inputs_dict['attention_mask'])
  inputs['labels'].append(label)

In [9]:
import pandas as pd
from tqdm import tqdm
import json
def parse_data(path, data_type='train'):
  sentence_a = []
  sentence_b = []
  labels = []

  with open(path, 'r', encoding = 'utf8') as f:
    for line in tqdm(f.readlines(), desc=f'Reading {data_type} data'):
      line = json.loads(line)
      sentence_a.append(line['sentence1'])
      sentence_b.append(line['sentence2'])
      if data_type != 'test':
        labels.append(int(line['label']))
      else:
        labels.append(0)

  df = pd.DataFrame(zip(sentence_a, sentence_b, labels), columns = ['text_a', 'text_b', 'labels'])
  return df

In [10]:
from collections import defaultdict
def read_data(config, tokenizer):
  train_df = parse_data(config['train_file_path'], data_type = 'train')
  dev_df = parse_data(config['dev_file_path'], data_type = 'dev')
  test_df = parse_data(config['test_file_path'], data_type = 'test')

  data_df = {'train': train_df, 'dev': dev_df, 'test': test_df}
  processed_data = {}
  
  for data_type, df in data_df.items():
    inputs = defaultdict(list)

    for i, row in tqdm(df.iterrows(), desc=f'Preprocessing {data_type} data', total = len(df)):
      label = row[2]
      sentence_a, sentence_b = row[0], row[1]
      build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer)

    processed_data[data_type] = inputs
  
  return processed_data


In [11]:
data = read_data(config, tokenizer)

Preprocessing test data: 100%|██████████| 3861/3861 [00:02<00:00, 1422.23it/s]


In [12]:
class Collator:
  def __init__(self, max_seq_len, tokenizer):
    self.max_seq_len = max_seq_len
    self.tokenizer = tokenizer

  def pad_and_truncate(self, input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len):
    input_ids = torch.zeros((len(input_ids_list), max_seq_len), dtype = torch.long)
    token_type_ids = torch.zeros_like(input_ids)
    attention_mask = torch.zeros_like(input_ids)
    for i in range(len(input_ids_list)):
      seq_len = len(input_ids_list[i])
      if seq_len <= max_seq_len:
        input_ids[i, :seq_len] = torch.tensor(input_ids_list[i], dtype = torch.long)
        token_type_ids[i, :seq_len] = torch.tensor(token_type_ids_list[i], dtype = torch.long)
        attention_mask[i, :seq_len] = torch.tensor(attention_mask_list[i], dtype = torch.long)

      else:
        input_ids[i] = torch.tensor(input_ids_list[i][:max_seq_len - 1] + [self.tokenizer.sep_token_id], dtype = torch.long)
        token_type_ids[i] = torch.tensor(token_type_ids_list[i][:max_seq_len], dtype = torch.long)
        attention_mask[i] = torch.tensor(attention_mask_list[i][:max_seq_len], dtype = torch.long)

    labels = torch.tensor(labels_list, dtype = torch.long)
    return input_ids, token_type_ids, attention_mask, labels

  def __call__(self, examples):
    input_ids_list, token_type_ids_list, attention_mask_list, labels_list = list(zip(*examples))
    cur_max_seq_len = max(len(input_ids) for input_ids in input_ids_list)
    max_seq_len = min(cur_max_seq_len, self.max_seq_len)

    input_ids, token_type_ids, attention_mask, labels = self.pad_and_truncate(input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len)

    data_dict = {
        'input_ids':input_ids,
        'token_type_ids':token_type_ids,
        'attention_mask':attention_mask,
        'labels':labels
    }
    return data_dict

In [13]:
collate_fn = Collator(config['max_seq_len'], tokenizer)

In [14]:
from torch.utils.data import Dataset
class AFQMCDataset(Dataset):
  def __init__(self, data_dict):
    super(AFQMCDataset, self).__init__()
    self.data_dict = data_dict

  def __getitem__(self, idx):
    data = (self.data_dict['input_ids'][idx],
         self.data_dict['token_type_ids'][idx],
         self.data_dict['attention_mask'][idx],
         self.data_dict['labels'][idx])
    return data
  
  def __len__(self):
    return len(self.data_dict['input_ids'])

In [15]:
from torch.utils.data import DataLoader, RandomSampler
def build_dataloader(config, data, tokenizer):
  train_dataset = AFQMCDataset(data['train'])
  dev_dataset = AFQMCDataset(data['dev'])
  test_dataset = AFQMCDataset(data['test'])

  # 使用桶采样
  if config['use_bucket']:
    # 基采样器RandomSampler
    train_sampler = RandomSampler(train_dataset)
    # drop_last 最后一个batch小于size 丢弃
    bucket_sampler = BucketBatchSampler(train_sampler, batch_size = config['batch_size'],
                      drop_last = False, sort_key = lambda x: len(train_dataset[x][0]),# 以 input_id 长度作为排序的指标
                      bucket_size_multiplier = config['bucket_multiplier'])
    train_dataloader = DataLoader(dataset = train_dataset, batch_sampler = bucket_sampler, num_workers = 4, collate_fn = collate_fn)
  else:
    train_dataloader = DataLoader(dataset = train_dataset, batch_size = config['batch_size'], shuffle = True, num_workers = 4, collate_fn = collate_fn)


  dev_dataloader = DataLoader(dataset = dev_dataset, batch_size = config['batch_size'], shuffle = False, num_workers = 4, collate_fn = collate_fn)
  test_dataloader = DataLoader(dataset = test_dataset, batch_size = config['batch_size'], shuffle = False, num_workers = 4, collate_fn = collate_fn)

  return train_dataloader, dev_dataloader, test_dataloader

In [16]:
train_dataloader, dev_dataloader, test_dataloader = build_dataloader(config, data, tokenizer)

NameError: ignored